# Jupyter Notebook UI to graph your TAP data!

Version 1.3 - Joseph Liang, Rankin Lab
Updated:
1. Upgraded folder path selection application
2. Upgraded dataset management (less moving parts for end-user)
3. output changed from tif -> png

## Known bug: Step 2 an empty windows displays in Mac. May also apply to linux/windows.

## Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

## 3. Run all cells/steps sequentially, even the ones that do not need input

## Steps that require input: #3, #6.1, #7

# 1. Importing Packages Required (No input required, just run)

In [ ]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import os #<- package used to work with system file paths
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- another package used to plot graphs
from itertools import cycle #<- package used to iterate down rows (used in step 5 to add tap column)
import ipywidgets as widgets #<- widget tool to generate button and tab for graphs
from IPython.display import display #<- displays widgets
from tkinter import Tk, filedialog #<- Tkinter is a GUI package
print("done step 1")

# 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

## Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

## An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)

In [ ]:
### Select Folder App - After you run, you can select your folder for filepath
button = widgets.Button(description = 'Select Folder') #<- Creates a button variable
display(button) #<- displays this button on output
def select_folder(b): #<- This is an action. Requires a variable, so I put in an arbitrary one 'b'
    global folder_path #<- sets a variable as a global variable, not just within this action
    Tk().withdraw() #<- Tkinter likes to create annoying empty windows. This removes them
    folder_path = filedialog.askdirectory() #<- Opens up a file explorer window, and determines folder path
    Tk().update()#<- below
    Tk().destroy()#<- this and the line above it removes the file explorer window after a selection is made
    print(folder_path) #<- this helps confirm that this action was performed
    print('done step 2')
button.on_click(select_folder) #<- describes what happens when you click on this button

# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Number of taps is pretty self-explanatory. How any taps does your experiment have? put in that number

This may be a bit confusing, but this is due to some coding syntax that you don't have to worry about.

3.2: Put in your date and experiment information - this will be reflected in your saved figure file name

In [ ]:

number_of_stimuli = 11 ###<------ stimuli in your experiment.

number_taps = range(1, number_of_stimuli + 1)  #<- do not change this


# if you have different ISIs in the same folder, then come back and change this 
# when you are graphing for the second set of data with the other ISI 
# (Generally data from same ISIs are graphed together)
# If changing ISI mid-analysis, you can just skip straight to step 8 after running this cell again

experiment = 'sens' #<- 'dishab', 'sens' ... etc etc
date = 'put in date'

#You can pick your colours for the bar plot - for default I have black and blue
colours = ('black', 'blue')

print("done step 3")

# 4. Constructing Filelist From Source File/Select File (Just run)

In line 6, change your file suffix keyword ('.prv', or '.trv') depending on which stimuli data you want to analyse (.trv for tap, .prv for pulse)

In [ ]:
os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.prv'): # '.prv' for light pulse, '.trv' for tap
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

# 5. Process Data Function (Just Run)

In [ ]:
def ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    print(f'this strain/treatment has {Strain_N} plates') #<- will output as the first number
    DF_Total = pd.concat([pd.read_csv(f, sep=' ', skiprows = 4, header = None) for f in strain_filelist],
                      ignore_index=True) #<- imports your data files
    DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
    DF_Total = DF_Total.rename( #<- more cleaning
                {0:'time',
                2:'rev_before',
                3:'no_rev',
                4:'stim_rev',
                7:'dist',
                8:'dist_std',
                9:'dist_stderr',
                11:'dist_0th',
                12:'dist_1st',
                13:'dist_2nd',
                14:'dist_3rd',
                15:'dist_100th',
                18:'dura',
                19:'dura_std',
                20:'dura_stderr',
                22:'dura_0th',
                23:'dura_1st',
                24:'dura_2nd',
                25:'dura_3rd',
                26:'dura_100th'}, axis=1)
    DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) #<- calculate prob
    DF_Total['speed'] = DF_Total['dist']/DF_Total['dura'] #<- calculate speed
    DF_Total["taps"] = np.nan #<- add a column called taps, the x axis for your graphs
    taps = cycle(number_taps) 
    DF_Total['taps'] = [next(taps) for count in range(DF_Total.shape[0])]
    DF_Total_rows = int(DF_Total.shape[0])
    print(f'this strain/treatment has {DF_Total_rows} total taps') #<- Outputs as the second number. Check if you are missing taps!
    DF_Final = DF_Total[["taps", "dura", "dist", "prob", "speed"]].copy() #<- Finally, clean data!
    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Final': DF_Final}
print('done step 5')

# 6a. Process Data (PLEASE READ, Add input here)

Here is the hardest part - From your naming convention, you need to pick a unique identifier for each group.

This means that all of names of your files for that strain should have that in common but is not commone with across all other files! If you did a good job naming your files and following a good naming convention, this should be easy.

## Be careful and really look hard in your naming structure. Note you want an unique identifier in the entire file path for the same group of files. An easy mistake is to have the strain name in the overall folder name, in this case if you use your strain name as a keyword it would include all files in that folder!

For example, if all your N2 files have a certain pattern like "N2_5x4" in this following example:
'/Users/Joseph/Desktop/AVR14_10sISI_TapHab_0710_2019/N2/20190710_141740/N2_5x4_f94h20c_100s30x10s10s_C0710ab.trv'

Then you need to set that identifier for the strain keyword:
'Strain_1' = 'N2_5x4'

## Depending on how many strains you are running for comparison, you may need to add/delete some lines!

## You are not naming your data groups here, we have a step for that later!
## Here, you want to note down ALL the strains you have in the folder

If you have just 2 strains, add hashtags (#) in front of the lines you do not need.
If you need more strains, just add more lines, following the same format!

In [ ]:
#Format: 'Strain_#'' = 'unique_identifier'

### Make into dictionary
StrainNames = {
    'Strain_1' : '',  #<- each strain will be designated to a unique identifier here
    'Strain_2' : '',
    'Strain_3' : 'AQ2755_tap',
    'Strain_4' : 'AQ2755_notap',
    'Strain_5' : 'VG222_tap',
    'Strain_6' : 'VG222_notap',
    'Strain_7' : 'VG233_tap', #<- empty entries are for those hardcore trackers that tracks this many strains
    'Strain_8' : 'VG233_notap',
    'Strain_9' : 'VG234_tap',
    'Strain_10' : 'VG234_notap',
    'Strain_11' : '',
    'Strain_12' : '',
    'Strain_13' : '',
    'Strain_14' : '',
    'Strain_15' : '',}
#...etc, etc

print('done step 6a')

# 6b. Process Data (just run this cell)

In [ ]:
DataLists = [0]  #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in StrainNames.values(): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
        DataLists.append(ProcessData(s)['Final']) #<- appends all data into a list of dataframes

print('done step 6b')

# 7. Grouping Data and Naming (Optional: Add input here)

Here, you get to name your data groups/strain! Name your groups however you like under between the quotation marks for each strain.

For example: If your Strain1 is N2 and you wish for the group to be called N2,
your line should look like:

DataLists[x].assign(dataset = 'N2')

## Go back to step 6.1 to check which strain is which item on the DataLists.
In this example, the first item on DataLists is N2.


## Remember: Put your name in quotes. (ex: 'N2' and not N2)

As default, the names are set to the unique identifier labels.

## Depending on the number of strains you are running the comparison, you may have to delete/add lines of code (following the same format). 
## Remember to add/delete commas too.

# If you want to change your groups, you do that here. 
For example, if you have 5 strains in your folder but only want to compare between 2 or 3 strains, designate that here and follow through with steps 6 and 7. Once you are done, come back to step 6 and change your groups again (You are going to have to change your graph titles for the second run-through though)!

# 7 - Run this if you are planning on doing a point plot

In [ ]:
TotalConcatenated = pd.concat([ #<- this function joins your data together with an extra column for which group 
                          DataLists[1].assign(dataset = StrainNames.get("Strain_1")), 
                          DataLists[2].assign(dataset = StrainNames.get("Strain_2")),
                          DataLists[3].assign(dataset = StrainNames.get("Strain_3")),
                          DataLists[4].assign(dataset = StrainNames.get("Strain_4")),
                          DataLists[5].assign(dataset = StrainNames.get("Strain_5")),
])
# ...etc etc
print(TotalConcatenated)
print('done step 7')

# 8. Graph Habituation Data - multiple stimuli (Probability, Distance, Duration, Speed of Tap Habituation POINT PLOT)

## Here, feel free to change the graph title and axis labels! Images are automatically saved in your folder with your data!

Note: It has been agreed by the lab that distance is quite an outdated measure, and the three top measures we consider are:

## probability, duration and speed.

In [ ]:
out1 = widgets.Output() #<- this is part of the code that helps display your graphs in tabs
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3, out4]) #<- more code to support tab display
tab.set_title(0, 'Probability')
tab.set_title(1, 'Distance')
tab.set_title(2, 'Duration')
tab.set_title(3, 'Speed')
display(tab)


# Probability
with out1:
    plt.figure(linewidth = 2.5) #<- Make your figure lines THICCCC for clean display (posters)
    plt.figure(figsize=(12,10)) #<- set your figure size (width, length)
    plt.gca().xaxis.grid(False) #<- gets rid of x-axis markers to make data look clean 
    ax = sns.pointplot(x="taps", #<- Here we use seaborn as our graphing package. 
                 y="prob",  
                 data = TotalConcatenated,
                 hue = 'dataset', #<- Here we use the extra column from step 6 to separate by group
                 palette = 'deep', #<- Change colour palette here if you like
                 ci = 95) #<- Confidence interval. 95 = standard error
    plt.xlabel("Taps") #<- X-axis title
    plt.ylabel("Probability") #<- Y-Axis title
    plt.title(f"Habituation Probability of {experiment}") #<- Figure Title
    ax.legend(loc = 'upper right', fontsize = '12') #<- location of your legend
    plt.savefig(f'Habituation_Probability_{experiment}.png', format='png', dpi=900) #<- saves your file to your folder
    plt.show()

# Distance
with out2:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dist", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = 'deep',
                 ci = 95)
    plt.xlabel("Taps")
    plt.ylabel("Distance")
    plt.title(f"Habituation Distance of {experiment}")
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Habituation_Distance_{experiment}.png', format='png', dpi=900)
    plt.show()

# Duration
with out3:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dura", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = 'deep',
                 ci = 95)
    plt.xlabel("Taps")
    plt.ylabel("Duration")
    plt.title(f"Habituation Duration of {experiment}")
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Habituation_Duration_{experiment}.png', format='png', dpi=900)
    plt.show()

# Speed
with out4:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="speed", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = 'deep',
                 ci = 95)
    plt.xlabel("Taps")
    plt.ylabel("Speed")
    plt.title(f"Habituation Speed of {experiment}")
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Habituation_Speed_{experiment}.png', format='png', dpi=900)
    plt.show()

# 9. Graph Sensitization/Dishabituation Data - last stimuli (Probability, Distance, Duration, Speed of Tap Habituation/Dishabituation/Sensitization BAR PLOT)

# 9a - Run this if you are planning on doing a bar plot

This is very much like step 7 - except from your naming convention it automatically extracts the name of the strain and the treatment - no need to add change anything!

In [ ]:
TotalConcatenatedbar = pd.concat([ #<- this function joins your data together with an extra column for which group 
                          DataLists[1].assign(strain = StrainNames.get("Strain_9").split('_')[0]).assign(stim = StrainNames.get('Strain_9').split('_')[1]), 
                          DataLists[2].assign(strain = StrainNames.get("Strain_10").split('_')[0]).assign(stim = StrainNames.get('Strain_10').split('_')[1]),
                          DataLists[3].assign(strain = StrainNames.get("Strain_3").split('_')[0]).assign(stim = StrainNames.get('Strain_3').split('_')[1]),
                          DataLists[4].assign(strain = StrainNames.get("Strain_4").split('_')[0]).assign(stim = StrainNames.get('Strain_4').split('_')[1]),
                          DataLists[5].assign(strain = StrainNames.get("Strain_5").split('_')[0]).assign(stim = StrainNames.get('Strain_5').split('_')[1]),
                          DataLists[6].assign(strain = StrainNames.get("Strain_6").split('_')[0]).assign(stim = StrainNames.get('Strain_6').split('_')[1]),
                          DataLists[7].assign(strain = StrainNames.get("Strain_7").split('_')[0]).assign(stim = StrainNames.get('Strain_7').split('_')[1]),
                          DataLists[8].assign(strain = StrainNames.get("Strain_8").split('_')[0]).assign(stim = StrainNames.get('Strain_8').split('_')[1])
])
# ...etc etc
print(TotalConcatenatedbar)
print('done step 9a')

# 9b. (Situation Dependent) - Only to graph last point of multi-simuli data

Here, if you only wish to bar plot dishabituation data (from a habituation/dishabituation experiment), you need to run the following cell first - the following filters out the lasts stimulus in the experiment (your dishabituation stimulus).

In [ ]:
TotalConcatenatedbar = TotalConcatenatedbar[TotalConcatenatedbar.taps == 10]
TotalConcatenatedbar
print('done 9b')

# 9c. Plotting

## Here, feel free to change the graph title and axis labels! Images are automatically saved in your folder with your data!

Note: It has been agreed by the lab that distance is quite an outdated measure, and the three top measures we consider are:

## probability, duration and speed.

In [ ]:
out1 = widgets.Output() #<- this is part of the code that helps display your graphs in tabs
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3, out4]) #<- more code to support tab display
tab.set_title(0, 'Probability')
tab.set_title(1, 'Distance')
tab.set_title(2, 'Duration')
tab.set_title(3, 'Speed')
display(tab)


# Probability
with out1:
    plt.figure(linewidth = 2.5) #<- Make your figure lines THICCCC for clean display (posters)
    plt.figure(figsize=(12,10)) #<- set your figure size (width, length)
    plt.gca().xaxis.grid(False) #<- gets rid of x-axis markers to make data look clean 
    ax = sns.barplot(x="strain", #<- Here we use seaborn as our graphing package. 
                 y="prob",  
                 data = TotalConcatenatedbar,
                 hue = 'stim', #<- Here we use the extra column from step 6 to separate by group
                 palette = colours, #<- Change colour palette here if you like
                 ci = 95) #<- Confidence interval. 95 = standard error
    plt.xlabel("Taps") #<- X-axis title
    plt.ylabel("Probability") #<- Y-Axis title
    plt.title(f"Probability of {experiment}") #<- Figure Title
    ax.legend(loc = 'upper right', fontsize = '12') #<- location of your legend
    plt.savefig(f'Bar_Probability_{experiment}_{date}.png', format='png', dpi=900) #<- saves your file to your folder
    plt.show()

# Distance
with out2:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.barplot(x="strain", 
                 y="dist", 
                 data = TotalConcatenatedbar,
                 hue = 'stim',
                 palette = colours,
                 ci = 95)
    plt.xlabel("Taps")
    plt.ylabel("Distance")
    plt.title(f"Distance of {experiment}")
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Bar_Distance_{experiment}_{date}.png', format='png', dpi=900)
    plt.show()

# Duration
with out3:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.barplot(x="strain", 
                 y="dura", 
                 data = TotalConcatenatedbar,
                 hue = 'stim',
                 palette = colours,
                 ci = 95)
    plt.xlabel("Taps")
    plt.ylabel("Duration")
    plt.title(f"Duration of {experiment}")
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Bar_Duration_{experiment}_{date}.png', format='png', dpi=900)
    plt.show()

# Speed
with out4:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.barplot(x="strain", 
                 y="speed", 
                 data = TotalConcatenatedbar,
                 hue = 'stim',
                 palette = colours,
                 ci = 95)
    plt.xlabel("Taps")
    plt.ylabel("Speed")
    plt.title(f"Speed of {experiment}")
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Bar_Speed_{experiment}_{date}.png', format='png', dpi=900)
    plt.show()